# Data Cleaning 

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession 

In [28]:
pyspark = SparkSession.builder \
.master("local[4]")\
.appName("DataCleaning")\
.config("spark.executer.memory","3g")\
.config("spark.driver.memory","3g")\
.getOrCreate()

In [29]:
sc = pyspark.sparkContext

#### Loading train dataset

In [5]:
train_data_df = spark.read\
.option("header", "True")\
.option("inferSchema", "True")\
.option("sep", ",")\
.csv("data/adult.data")

#### Loading test dataset

In [6]:
test_data_df = spark.read\
.option("header", "True")\
.option("inferSchema", "True")\
.option("sep", ",")\
.csv("data/adult.test")

## Data Joining

1. All empty row shoul be checked
2. (.) point sign must be cleaned for <=50K. and >50K. outputs.
3. Checking of NULL values in every column.
4. ? sign containing workclass, occupation and native_country columns should be investigated again.  ? sign should be investigated whether is systemically error or random error.
5. never-worked and without-pay classes in workclass and Armed-Forces class in occupation are less repetitive classes. They can be removed. We decide to remove these classes. Also Holand country repetitive number is 1, so that it had better to removed.
6. In education column has lot of categories. They should be better to combine some columns into one column.
   
   1st-4th, 5th-6th, 7th-8th ---> elemantary-school

    9th, 10th, 11th, 12th ---> high-school

    Masteres, Doctorate ---> postgraduate

    Bachelors, Some-college ---> undergraduate

In [7]:
whole_df = train_data_df.union(test_data_df)
whole_df.limit(5).toPandas().head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,output
0,39,State-gov,77516.0,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,United-States,<=50K
1,50,Self-emp-not-inc,83311.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K
2,38,Private,215646.0,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K
3,53,Private,234721.0,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50K
4,28,Private,338409.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K


### 1. All empty rows should be checked

In [8]:
from pyspark.sql.functions import *

In [9]:
emptyRow_df = whole_df\
.withColumn("workclass", trim(col("workclass")))\
.withColumn("education", trim(col("education")))\
.withColumn("marital_status", trim(col("marital_status")))\
.withColumn("occupation", trim(col("occupation")))\
.withColumn("relationship", trim(col("relationship")))\
.withColumn("race", trim(col("race")))\
.withColumn("sex", trim(col("sex")))\
.withColumn("native_country", trim(col("native_country")))\
.withColumn("output", trim(col("output")))

In [10]:
print("Before trim df: ",whole_df.count())
print("After trim  df: ",emptyRow_df.count())

Before trim df:  48842
After trim  df:  48842


### 2. Cleaning of points in output column

In [11]:
cleanPoints_df = emptyRow_df\
.withColumn("output", regexp_replace(col("output"), "<=50K.", "<=50K"))\
.withColumn("output", regexp_replace(col("output"), ">50K.", ">50K"))

In [12]:
cleanPoints_df.groupBy(col("output")).agg({"*":"count"}).show()

+------+--------+
|output|count(1)|
+------+--------+
| <=50K|   37155|
|  >50K|   11687|
+------+--------+



### 3. Checking of NULL values

In [13]:
emptyRow_df.columns

['age',
 'workclass',
 'fnlwgt',
 'education',
 'education_num',
 'marital_status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'capital_gain',
 'capital_loss',
 'hours_per_week',
 'native_country',
 'output']

In [14]:
count_for_null = 1
for column in cleanPoints_df.columns:
    if(cleanPoints_df.filter(col(column).isNull()).count()>0):
        print(count_for_null, ".", column, "--> \033[1m there has null values \033[0m")
    else:
        print(count_for_null, ".",column,"--> is clean")
    count_for_null += 1

1 . age --> is clean
2 . workclass --> is clean
3 . fnlwgt --> is clean
4 . education --> is clean
5 . education_num --> is clean
6 . marital_status --> is clean
7 . occupation --> is clean
8 . relationship --> is clean
9 . race --> is clean
10 . sex --> is clean
11 . capital_gain --> is clean
12 . capital_loss --> is clean
13 . hours_per_week --> is clean
14 . native_country --> is clean
15 . output --> is clean


[Interpretation]:  Here has worked else condition. It means that does not have any null value in dataset

### 4. Checking of "?" sign 

In [15]:
count_for_question = 1
for column in cleanPoints_df.columns:
    if(cleanPoints_df.filter(col(column).contains("?")).count() > 0):
        print(count_for_question,".",column, "--> \033[1m there has ? sign. \033[0m")
    else:
        print(count_for_question,".",column,"\033[0m--> is clean")
    count_for_question += 1

1 . age --> is clean
2 . workclass -->  there has ? sign. 
3 . fnlwgt --> is clean
4 . education --> is clean
5 . education_num --> is clean
6 . marital_status --> is clean
7 . occupation -->  there has ? sign. 
8 . relationship --> is clean
9 . race --> is clean
10 . sex --> is clean
11 . capital_gain --> is clean
12 . capital_loss --> is clean
13 . hours_per_week --> is clean
14 . native_country -->  there has ? sign. 
15 . output --> is clean


[Interpretation]:  The 3 questions were found. After founding questions (?), we need to check did they appear systematically or randomly. So that we need to make second analysis.

workclass, occupation, native_country have question (?) signs.

#### Here we filtered if ? sign contains in workclass or occupation or native_country
Relationship is investigated between these 3 variables and output values.

In [16]:
cleanPoints_df.select("workclass","occupation","native_country","output")\
.filter(
    col("workclass").contains("?") |
    col("occupation").contains("?") |
    col("native_country").contains("?")
)\
.groupBy("workclass","occupation","native_country","output").count()\
.orderBy(col("count").desc())\
.toPandas().head(20)

,workclass,occupation,native_country,output,count
0,?,?,United-States,<=50K,2284
1,?,?,United-States,>50K,246
2,Private,Other-service,?,<=50K,100
3,Private,Sales,?,<=50K,55
4,Private,Prof-specialty,?,<=50K,51
5,Private,Craft-repair,?,<=50K,48
6,Private,Prof-specialty,?,>50K,48
7,?,?,Mexico,<=50K,48
8,Private,Adm-clerical,?,<=50K,47
9,Private,Machine-op-inspct,?,<=50K,42


Systematic Error: the error source is known and can be corrected. There must absolutely be have a rule for systematic error. If you repeat the experiment, you’ll get the same error.
Example: If weight of 0  measurement reads as a 5 grams, 10 grams will read 15 gram. So that after measurement we always substract 5 gram from result.

Random Error: the error source is not known and can't be corrected. The random errors decrease the reliability. They are unpredictable and can’t be replicated by repeating the experiment again.
Example: If you measure the weight of apple five times using the same scales and get slightly different values (50gr, 51gr, 50,6gr). 

[Interpretation]: Relationship between question signs and output values are randomly. Between Workclass and Occupation have relationship. Because when workclass is ?, also occupation is ?. Therefore when workclass is not known  occupation is not known too. Consequently we decide to remove all rows which three of columns have question sign.

In [17]:
questionClean_df = cleanPoints_df\
.filter(~(
    col("workclass").contains("?") |
    col("occupation").contains("?")|
    col("native_country").contains("?")))

In [18]:
print("Before question cleaning: ",cleanPoints_df.count())
print("After  question cleaning: ",questionClean_df.count())
deleted_row = cleanPoints_df.count()-questionClean_df.count()
print("Removed row number: ", deleted_row)

Before question cleaning:  48842
After  question cleaning:  45222
Removed row number:  3620


### 5. Removing weak Class 

Less repetitive classes can be removed. In workclass never-worked and without-pay classes and In occupation column Armed-Forces class are less repetitive, so that they can be removed from dataset.

In [19]:
weakClean_df = questionClean_df\
.filter(~(
    col("workclass").contains("never-worked") |
    col("workclass").contains("without-pay") |
    col("occupation").contains("Armed-Forces")
))

In [20]:
print("Before removing weak classes: ", questionClean_df.count())
print("After  removing weak classes: ", weakClean_df.count())
deleted_weak_row = questionClean_df.count() - weakClean_df.count()
print("Removed row number: ", deleted_weak_row)

Before removing weak classes:  45222
After  removing weak classes:  45208
Removed row number:  14


### 6. Combining of categories about education statuses 

#### Listing of education categories (it is more simple than before)

In [21]:
whole_df.groupBy(col("education")).agg({"*":"count"}).select("education").toPandas().head(20)

,education
0,Prof-school
1,10th
2,7th-8th
3,5th-6th
4,Assoc-acdm
5,Assoc-voc
6,Masters
7,12th
8,Preschool
9,9th


We have 15 education category types. We combine some of them in one category as follows. 

1st-4th, 5th-6th, 7th-8th ---> elemantary-school

  9th, 10th, 11th, 12th   ---> high-school
  
   Masteres, Doctorate   ---> postgraduate
    
  Bachelors, Some-college ---> undergraduate
  

In [22]:
categoryCombine_df = weakClean_df.withColumn("education_merged",
when(col("education").isin("1st-4th", "5th-6th", "7th-8th"), "Elemantary-School")
.when(col("education").isin("9th", "10th", "11th", "12th"), "High-School")
.when(col("education").isin("Masters", "Doctorate"), "Postgraduate")
.when(col("education").isin("Bachelors", "Some-college"), "Undergraduate")
.otherwise(col("education")))

#### Listing of education categories after combining

In [23]:
categoryCombine_df.select("education", "education_merged").toPandas().head(10)

,education,education_merged
0,Bachelors,Undergraduate
1,Bachelors,Undergraduate
2,HS-grad,HS-grad
3,11th,High-School
4,Bachelors,Undergraduate
5,Masters,Postgraduate
6,9th,High-School
7,HS-grad,HS-grad
8,Masters,Postgraduate
9,Bachelors,Undergraduate


## --> Writing the cleaned dataset to disc 

#### Selecting of columns 

In [24]:
categoryCombine_df.toPandas().head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,output,education_merged
0,39,State-gov,77516.0,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,United-States,<=50K,Undergraduate
1,50,Self-emp-not-inc,83311.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K,Undergraduate
2,38,Private,215646.0,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K,HS-grad
3,53,Private,234721.0,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50K,High-School
4,28,Private,338409.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K,Undergraduate


In [25]:
attribute_order = ["workclass", "education", "education_merged", "marital_status", "occupation", "relationship", "race", "sex", 
        "native_country", "age", "fnlwgt", "education_num", "capital_gain", "capital_loss", "hours_per_week","output"]

clean_df = categoryCombine_df.select(attribute_order)
clean_df.toPandas().head()

,workclass,education,education_merged,marital_status,occupation,relationship,race,sex,native_country,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week,output
0,State-gov,Bachelors,Undergraduate,Never-married,Adm-clerical,Not-in-family,White,Male,United-States,39,77516.0,13.0,2174.0,0.0,40.0,<=50K
1,Self-emp-not-inc,Bachelors,Undergraduate,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,50,83311.0,13.0,0.0,0.0,13.0,<=50K
2,Private,HS-grad,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,United-States,38,215646.0,9.0,0.0,0.0,40.0,<=50K
3,Private,11th,High-School,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,United-States,53,234721.0,7.0,0.0,0.0,40.0,<=50K
4,Private,Bachelors,Undergraduate,Married-civ-spouse,Prof-specialty,Wife,Black,Female,Cuba,28,338409.0,13.0,0.0,0.0,40.0,<=50K


In [26]:
clean_df.count()

45208

#### Writing cleaned dataframe to disc

In [27]:
clean_df\
.coalesce(1)\
.write\
.mode("overwrite")\
.option("sep", ",")\
.option("header", "True")\
.csv("data/cleaned_dataset")